In [12]:
import os
import numpy as np
import pandas as pd

###Step1. 파일을 읽어옵니다.

In [3]:
spine_raw = pd.read_excel("DXA_meta_included.xlsx", sheet_name='spine', dtype={'patientID':str})
left_raw = pd.read_excel("DXA_meta_included.xlsx", sheet_name='left_femur', dtype={'patientID':str})
right_raw = pd.read_excel("DXA_meta_included.xlsx", sheet_name='right_femur', dtype={'patientID':str}) 

###Step2. 중복값 구분
각 sheet마다 patientID와 StudyDate를 기준으로 하여 중복값이 있는 그룹과 없는 그룹으로 구분합니다.

In [76]:
spine_no_dup = pd.DataFrame()
spine_yes_dup = pd.DataFrame()
for key, group in spine_raw.groupby(['patientID', 'StudyDate_x']):
    if len(group)==1:
        spine_no_dup = pd.concat([spine_no_dup, group])
    else:
        spine_yes_dup = pd.concat([spine_yes_dup, group])

In [72]:
left_no_dup = pd.DataFrame()
left_yes_dup = pd.DataFrame()
for key, group in left_raw.groupby(['patientID', 'StudyDate_x']):
    if len(group)==1:
        left_no_dup = pd.concat([left_no_dup, group])
    else:
        left_yes_dup = pd.concat([left_yes_dup, group])

In [9]:
right_no_dup = pd.DataFrame()
right_yes_dup = pd.DataFrame()
for key, group in right_raw.groupby(['patientID', 'StudyDate_x']):
    if len(group)==1:
        right_no_dup = pd.concat([right_no_dup, group])
    else:
        right_yes_dup = pd.concat([right_yes_dup, group])

In [77]:
#'-'을 nan 값으로 교체 해줍니다.
spine_yes_dup = spine_yes_dup.replace('-', np.nan)
left_yes_dup = left_yes_dup.replace('-', np.nan)
right_yes_dup = right_yes_dup.replace('-', np.nan)

###Step3. 중복값 제거
중복값 제거 기준<br>
<dl>
  <dt>- 모든 컬럼값이 동일한 경우: 랜덤으로 1개 레코드 선택
  <dt>- 그렇지 않은 경우:
    <dd>NULL개수가 차이가 나는 경우: NULL개수가 적은 레코드 선택</dd>
    <dd>NULL개수가 차이가 없는 경우: 랜덤으로 1개 레코드 선택</dd>
</dl> 

In [78]:
spine_drop_dup = pd.DataFrame()
for key, group in spine_yes_dup.groupby(['patientID','StudyDate_x']):
    g = group[group.columns[20:]]
    g_no_dup = g.drop_duplicates()

    if len(g_no_dup) == 1: #record가 완전 중복, 랜덤하게 하나 선택
        spine_drop_dup = pd.concat([spine_drop_dup, group.head(1)])
    else: # Nan이 적은 row로 랜덤하게 하나 선택
        min_nan_count = min(g.isna().sum(axis=1).tolist())
        select = g[g.isna().sum(axis=1) == min_nan_count]
        spine_drop_dup = pd.concat([spine_drop_dup, group.loc[[select.head(1).index[0]]]])

In [74]:
left_drop_dup = pd.DataFrame()
for key, group in left_yes_dup.groupby(['patientID','StudyDate_x']):
    g = group[group.columns[20:]]
    g_no_dup = g.drop_duplicates()

    if len(g_no_dup) == 1: #record가 완전 중복, 랜덤하게 하나 선택
        left_drop_dup = pd.concat([left_drop_dup, group.head(1)])
    else: # Nan이 적은 row로 랜덤하게 하나 선택
        min_nan_count = min(g.isna().sum(axis=1).tolist())
        select = g[g.isna().sum(axis=1) == min_nan_count]
        left_drop_dup = pd.concat([left_drop_dup, group.loc[[select.head(1).index[0]]]])

In [68]:
right_drop_dup = pd.DataFrame()
for key, group in right_yes_dup.groupby(['patientID','StudyDate_x']):
    g = group[group.columns[20:]]
    g_no_dup = g.drop_duplicates()

    if len(g_no_dup) == 1: #record가 완전 중복, 랜덤하게 하나 선택
        right_drop_dup = pd.concat([right_drop_dup, group.head(1)])
    else: # Nan이 적은 row로 랜덤하게 하나 선택
        min_nan_count = min(g.isna().sum(axis=1).tolist())
        select = g[g.isna().sum(axis=1) == min_nan_count]
        right_drop_dup = pd.concat([right_drop_dup, group.loc[[select.head(1).index[0]]]])
        


###Step4. 내보내기
정리된 데이터들을 엑셀파일로 내보냅니다.

In [79]:
spine_final = pd.concat([spine_no_dup, spine_drop_dup])
# left_final = pd.concat([left_no_dup, left_drop_dup])
# right_final = pd.concat([right_no_dup, right_drop_dup])

In [83]:
with pd.ExcelWriter('SNUBH.xlsx') as o:
    spine_final.to_excel(o, sheet_name='spine', index=False)
    left_final.to_excel(o, sheet_name='left_femur', index=False)
    right_final.to_excel(o, sheet_name='right_femur', index=False)

###Appendix: 확인하기
patientID와 StudyDate를 키로 주었을 때 row가 unique한지 확인해봅니다.

In [89]:
len(right_final)

814

In [88]:
len(right_final[['patientID', 'StudyDate_x']].drop_duplicates())

814